In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tqdm

# Prepare Data

In [2]:
# 將WORD轉成INDEX
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}

# 將所需要的WORD加入DICTIONARY中
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
# 定義EMBEDDING 的維度與HIDDEN LAYER DIMENSION的大小
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


# Define the network structure

In [3]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        #EMBEDDING 的層要決定文字量與DIMENSIONS
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        # LSTM 輸入 = EMBEDDING_DIMENSION + 決定要有幾格OUTPUT
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        # 輸入為LSTM最後一個TIME STEP的OUTPUT及要預測的CLASS數量
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
        # LSTM要初始化每一層NEURON的參數
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        #初始化要輸入(幾層LSTM, BATCH_SIZE, 幾層隱藏層)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        '''
        1. 先將文字EMBEDDING
        2. 將EMBEDDING 後的文字帶入LSTM
        3. 將LSTM的OUTPUT帶入FULLLY CONECTTED 層
        4. 輸出的OUTPUT在過LOG(SOFTMAX)作為最後結果
        '''
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Start To Training

In [5]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

#定義OPTIMIZER 與 LOSS FUNCTION
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# 先顯示沒有更新GRADIENT的OUTPUT結果
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

# 開始訓練
for epoch in tqdm.tqdm_notebook(range(300)):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        # 先將GRADIENT 初始化
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        # 將LSTM的NEURON 初始化
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        # 將文字轉成INDEX並轉成TENSOR
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        # 帶入模型當中
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        # 計算LOSS 與 UPDATE GRADIENTS
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.8166, -1.0555, -1.5605],
        [-0.8598, -1.0077, -1.5526],
        [-0.8501, -1.0405, -1.5171],
        [-0.7766, -1.1020, -1.5711],
        [-0.7706, -1.1204, -1.5554]])


HBox(children=(IntProgress(value=0, max=300), HTML(value='')))


tensor([[-0.0376, -3.6371, -4.5483],
        [-5.7804, -0.0342, -3.4877],
        [-4.5756, -3.0479, -0.0595],
        [-0.0122, -4.9974, -5.2301],
        [-4.5540, -0.0386, -3.5995]])
